
**Building a RAG pipeline with Llamaindex**





In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
import os
print(os.path.exists('/content/drive/MyDrive/'))


True


load pdf from local Drive

In [4]:
!ls /content/drive/MyDrive/diotima

Book-GBV-GREEK_final.pdf


In [5]:
#pdf related to domestic violence - educational material from independent organization
DATA_DIR="/content/drive/MyDrive/diotima"

use of Llamaindex a framework focused towards efficient indexing and retrieval, providing multiple components for all the RAG steps.

In [6]:
!pip install llama-index llama-index-readers-file llama-index-llms-openai llama-index-embeddings-huggingface llama-index-llms-litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.llms.litellm import LiteLLM

In [8]:
documents = SimpleDirectoryReader(DATA_DIR).load_data()
len(documents)

86

In [9]:
print(len(documents))

86


chunking: semantic splitter

In [10]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

nodes = splitter.get_nodes_from_documents(documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [11]:
len(nodes)

187

In [12]:
nodes[0]

TextNode(id_='1dfe835f-9bf3-4e4f-9e94-42f2f9d149ea', embedding=None, metadata={'page_label': '1', 'file_name': 'Book-GBV-GREEK_final.pdf', 'file_path': '/content/drive/MyDrive/diotima/Book-GBV-GREEK_final.pdf', 'file_type': 'application/pdf', 'file_size': 6688819, 'creation_date': '2024-12-01', 'last_modified_date': '2024-12-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9e60ce84-17ef-422b-9c82-581bf88efd9d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Book-GBV-GREEK_final.pdf', 'file_path': '/content/drive/MyDrive/diotima/Book-GBV-GREEK_final.pdf', 'file_type': 'application/pdf', 'file_size': 6688819, 'creation_date': '2024-12-01', 'last_modified_date': '2024-12-

create vector store

In [13]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
index = VectorStoreIndex(nodes, embed_model=embed_model)

answers from LLM

In [14]:
# gpt35 = LLamaIndexOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
# meltemi = LLamaIndexOpenAI(api_key=MELTEMI_API_KEY, base_url=MELTEMI_BASE_URL, model="meltemi-instruct-7b")

meltemi = LiteLLM(
    "hosted_vllm/meltemi-vllm",
    api_base="http://ec2-3-19-37-251.us-east-2.compute.amazonaws.com:4000/",
    api_key="sk-RYF0g_hDDIa2TLiHFboZ1Q",
)

prompt localization for LlamaIndex

In [15]:
from llama_index.core.prompts import (
    SelectorPromptTemplate,
    PromptType,
    PromptTemplate,
    MessageRole,
    ChatPromptTemplate,
    ChatMessage,
)

QA_PROMPT = """BEGININPUT
{context_str}

ENDINPUT
BEGININSTRUCTION
{query_str}
ENDINSTRUCTION"""

QA_SYSTEM_PROMPT = """Είσαι ένας ειλικρινής και αμερόληπτος βοηθός που πάντα απαντάει με ακρίβεια σε αυτά που του ζητούνται.
Σου δίνεται ένα έγγραφο το οποίο βρίσκεται μεταξύ του BEGININPUT και του ENDINPUT.
Επίσης, σου δίνονται μεταδεδομένα για το συγκεκριμένο έγγραφο μεταξύ του BEGINCONTEXT και του ENDCONTEXT.
Το βασικό κείμενο του εγγράφου βρίσκεται μεταξύ του ENDCONTEXT και του ENDINPUT.
Απάντα στις οδηγίες του χρήστη που βρίσκονται μεταξύ του BEGININSTRUCTION και του ENDINSTRUCTION χρησιμοποιώντας μόνο το βασικό κείμενο
και τα μεταδεδομένα του εγγράφου που σου δίνονται παρακάτω. Αν οι οδηγίες που σου ζητάει ο χρήστης δεν μπορούν να απαντηθούν με το βασικό
κείμενο ή τα μεταδεδομένα του εγγράφου, ενημέρωσε τον χρήστη ότι δεν ξέρεις την σωστή απάντηση."""

REFINE_PROMPT = """Η αρχική ερώτηση είναι η εξής: {query_str}
Έχεις δώσει την παρακάτω απάντηση: {existing_answer}
Έχεις την ευκαιρία να βελτιώσεις την προηγούμενη απάντηση (μόνο αν χρειάζεται) με τις παρακάτω νέες πληροφορίες.
------------
{context_str}
------------
Με βάση τις νέες πληροφορίες, βελτίωσε την απάντησή σου για να απαντά καλύτερα στην ερώτηση.
Αν οι πληροφορίες δεν είναι χρήσιμες, απλά επανάλαβε την προηγούμενη απάντηση.
Βελτιωμένη απάντηση: """

REFINE_CHAT_PROMPT = """Είσαι ένα έμπιστο σύστημα που απαντάει ερωτήσεις χρηστών. Λειτουργείς με τους εξής δύο τρόπους όταν βελτιώνεις υπάρχουσες απαντήσεις:
1. **Ξαναγράφεις** την απάντηση με βάση νέες πληροφορίες που σου παρέχονται
2. **Επαναλαμβάνεις** την προηγούμενη απάντηση αν δεν είναι χρήσιμες οι νέες πληροφορίες

Ποτέ δεν αναφέρεις την αρχική απάντηση ή το συγκείμενο απευθείας στην απάντησή σου.
Αν υπάρχει αμφιβολία για το τι πρέπει να απαντήσεις, απλά επανέλαβε την αρχική απάντηση.
Νέες πληροφορίες:
------------
{context_str}
------------
Ερώτηση: {query_str}
Αρχική απάντηση: {existing_answer}
Βελτιωμένη απάντηση: """


default_template = PromptTemplate(
    metadata={"prompt_type": PromptType.QUESTION_ANSWER},
    # template_vars=["context_str", "query_str"],
    template=QA_PROMPT,
)
chat_template = ChatPromptTemplate(
    metadata={"prompt_type": PromptType.CUSTOM},
    # template_vars=["context_str", "query_str"],
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=QA_SYSTEM_PROMPT),
        ChatMessage(role=MessageRole.USER, content=QA_PROMPT),
    ],
)
text_qa_prompt = SelectorPromptTemplate(
    default_template=default_template, conditionals=[(lambda llm: True, chat_template)]
)

default_refine_template = PromptTemplate(
    metadata={"prompt_type": PromptType.REFINE},
    # template_vars=["query_str", "existing_answer", "context_str"],
    template=REFINE_PROMPT,
)
chat_refine_template = ChatPromptTemplate(
    metadata={"prompt_type": PromptType.CUSTOM},
    # template_vars=["context_str", "query_str", "existing_answer"],
    message_templates=[ChatMessage(role=MessageRole.USER, content=REFINE_CHAT_PROMPT)],
)
text_refine_prompt = SelectorPromptTemplate(
    default_template=default_refine_template,
    conditionals=[(lambda llm: True, chat_refine_template)],
)

In [16]:
query_engine = index.as_query_engine(
    llm=meltemi, text_qa_prompt=text_qa_prompt, text_refine_prompt=text_refine_prompt
)

In [17]:
response = query_engine.query("Ποια είναι τα είδη της έμφυλης βίας;")
print(str(response))

Η έμφυλη βία περιλαμβάνει οιαδήποτε επιβλαβή πράξη που στρέφεται ενάντια σε ένα άτομο με βάση το φύλο του, συμπεριλαμβανομένων των διαφορετικών μορφών εξαναγκασμού και/ή της εμπλοκής ανηλίκων. Αυτές οι πράξεις μπορεί να προκύψουν σε δημόσιο ή ιδιωτικό χώρο.

Βεβαιότητα: 90%


In [18]:
response = query_engine.query("Πού βρήκες την απάντηση που μου έδωσες;")
print(str(response))

Η απάντηση που μου δώσατε δεν είναι διαθέσιμη στο παρεχόμενο πλαίσιο.


In [20]:
response = query_engine.query("Τι πρέπει να αποφύγω όταν μιλάω με άτομο που επέζησε από έμφυλη βία")
print(str(response))

Όταν επικοινωνείτε με άτομα που επέζησαν από έμφυλη βία, είναι σημαντικό να αποφύγετε να κάνετε υποθέσεις, να κάνετε προσωπικές ερωτήσεις ή να κάνετε ερωτήσεις που μπορεί να είναι τραυματικές. Είναι επίσης σημαντικό να σεβαστείτε την ιδιωτικότητά τους και να μην αποκαλύψετε τις πληροφορίες σε άλλους χωρίς τη συγκατάθεσή τους. Επιπλέον, είναι σημαντικό να ακούσετε το άτομο και να του προσφέρετε υποστήριξη και πόρους, αντί να προσπαθίσετε να δώσετε συμβουλές ή να κάνετε δηλώσεις που μπορεί να είναι πατερναλιστικές ή να υποτιμούν την εμπειρία του.

Απαραίτητες συμβουλές:

• Ακούστε προσεκτικά: λέξεις, χροιά φωνής και εκφράσεις που χρησιμοποιούνται για ό,τι δεν λέγεται.
• ΠΡΟΣΟΧΗ!

Να θυμάστε, είναι σημαντικό να είστε υπομονετικοί, σεβαστοί και υποστηρικτικοί όταν επικοινωνείτε με άτομα που επέζησαν από έμφυλη βία. Είναι σημαντικό να ακούσετε το άτομο και να του προσφέρετε υποστήριξη και πόρους, αντί να προσπαθήσετε να δώσετε συμβουλές ή να κάνετε δηλώσεις που μπορεί να είναι πατερναλιστικ